In [71]:
import serial
import time
import pandas as pd
import scipy
from scipy import signal
import numpy as np
import math
from statistics import mean

In [72]:
import pickle
global clf
clf = pickle.load(open('testmodel.pkl','rb'))

In [74]:
def event_detection(data):
    '''
    Detects devices being turned On
    Returns the timestamp indices where devices are turned on
    '''
    data = scipy.signal.medfilt(data, kernel_size=None)
    dbydx = np.gradient(data)
    on_events = []
    for idx,x in enumerate(dbydx):
        if x > THRSHLD:
            on_events.append(idx)
    for x in on_events:
        for y in range(x+1,x+91): # to remove false turn on signatures, give delay for transition
            if y in on_events:
                on_events.remove(y)
    return on_events

In [75]:
def get_signal (length):
    '''
    Takes the length of signature required,
    returns signature and timestamps
    
    '''
    COUNT = 0
    buffer = np.array([])
    time_buffer = np.array([])
    while COUNT!= length :
        b = ser.readline()         # read a byte string
        string_n = b.decode(errors='ignore')  # decode byte string into Unicode  
        print(string_n)
        try:
            val, timestamp = string_n.split(sep = ',')
            val = float(val)
            timestamp = pd.to_datetime(timestamp)
            buffer = np.append(buffer, val) # helps getting data in seconds
            time_buffer = np.append(time_buffer,timestamp)
            COUNT += 1
        except:
            print('read failed, retrying!')
    return buffer,time_buffer

In [76]:
def predict_clf(data): #unused
    global clf
    guess = clf.predict(data)
    acc = clf.predict_proba(data).max()
    print(f'Device turned on at {sig_ts} and is probably {guess}, Acc.: {acc}') 

In [126]:
def log_event(guess,sig_ts,rise,sig):
    '''
    Stores the following:
    1. Signature of appliance,
    2. Time-stamp of turning ON
    3. Name
    4. Rise in consumption
    
    '''
    global logs_df
    logs_df = logs_df.append(pd.DataFrame({'Sig':[sig],
                                           'Name':guess,
                                           'Turned ON':sig_ts,
                                           'Rise':rise},index=[0]),ignore_index=True)

In [ ]:
global logs_df
logs_df = pd.DataFrame([],columns = ['Sig','Name','Turned ON','Rise'])

In [153]:
SIG_SIZE = 90 # 1 sec contains 30 samples ; 3 seconds 90 samples.
THRSHLD = 30 # Rise in kWH to be detected for classification
BUFFER_SIZE = 360 # signal buffer size 
i = 1 #test purpose iterator
prev_mean = 0 #maybe declare global
# These are for creating local database and saving signatures!!!
prev_guess=''

In [151]:
ser = serial.Serial('COM3', 115200) # remove later
time.sleep(2)


# Read and record the data
for i in range(10):
    buffer, time_buffer = get_signal(BUFFER_SIZE)
    events = event_detection(buffer)
    if len(events)>0:
        start = time.time()
        print(f'{len(events)} events detected')
        for ev in events:
            sig = buffer[ev:ev+SIG_SIZE]
            if len(sig) != SIG_SIZE:
                print('signal not adequate, shifting buffer')
                buffer = np.roll(buffer, -SIG_SIZE)
                time_buffer = np.roll(time_buffer, -SIG_SIZE)
                temp_buffer, temp_time_buffer = get_signal(SIG_SIZE)
                buffer[BUFFER_SIZE-SIG_SIZE:] = temp_buffer
                time_buffer[BUFFER_SIZE-SIG_SIZE:] = temp_time_buffer
                events = event_detection(buffer)
                print(f'{len(events)} events detected on reconstructing')
            if len(events)>0:
                for ev in events:
                    sig = buffer[ev:ev+SIG_SIZE]
                    if sig.mean() > prev_mean:
                        sig = sig - prev_mean
                        sig_ts = time_buffer[ev]
                        sig = sig.reshape((sig.shape[0]//SIG_SIZE,SIG_SIZE))
                        rise = buffer[ev+SIG_SIZE:].mean()
                        guess = clf.predict(sig)
                        end = time.time()
                        acc = clf.predict_proba(sig).max()
                        print(f'{guess} turned on at {sig_ts} with rise {rise}, Acc.: {acc}')
                        if acc > 0.80:
                            if prev_guess == guess:
                                print(f'It seems the {guess} was detected again at {sig_ts}, if true confirm in {local_name}, but if different, name it\n')
                                name = input('Enter the name!\n')
                                if name == guess:
                                    # for decremening active_devices and clubbing
                                    # save signature
                                    pass
                                else:
                                    # save signature
                                    pass
                                prev_guess = guess
                                log_event(name,sig_ts,rise,sig)
                                #here might want to decrement active_devices and remove start time from it and club it by comparing prev and current guess!
                            else:
                                name = input(f'An appliance  was detected at {sig_ts}! Some people named it as {guess}, would you like to name it?\n')
                                prev_guess = guess
                                log_event(name,sig_ts,rise,sig)
                        else:
                            name = input(f'An unrecognized device was detected at {sig_ts}, please name it\n')
                            prev_guess = guess
                            log_event(name,sig_ts,rise,sig)
                print(f'Total time taken {end-start:.3f} s')
                ser.close() # remove later
                break # breaks previous 'for-loops' of events 
        break #breaks i iterator

    else:
        prev_mean = buffer.mean()

SyntaxError: invalid syntax (<ipython-input-151-8d9bec97e181>, line 31)

In [152]:
logs_df

,Sig,Name,Turned ON,Rise
0,"[0.0, 97.0, 97.0, 97.0, 97.0, 97.0, 97.0, 97.0...",hairdryer,2020-11-03 23:06:14.710,300.752941
1,"[48.0, 289.0, 289.0, 289.0, 289.0, 289.0, 289....",hairdryer,2020-11-03 23:07:31.760,306.000000
2,"[12.0, 260.0, 260.0, 260.0, 260.0, 260.0, 260....",water heater,2020-11-03 23:09:59.685,298.968254
3,"[51.0, 279.0, 279.0, 279.0, 279.0, 279.0, 279....",water heater,2020-11-03 23:12:02.547,292.736842
4,"[49.0, 276.0, 276.0, 276.0, 276.0, 276.0, 276....",water heater,2020-11-03 23:16:43.377,290.141509
5,"[0.0, 210.0, 210.0, 210.0, 210.0, 210.0, 210.0...",water heater,2020-11-03 23:19:12.670,285.979866
6,"[49.0, 275.0, 275.0, 275.0, 275.0, 275.0, 275....",water heater,2020-11-03 23:20:18.299,287.741667
7,"[[1.0, 119.0, 119.0, 119.0, 119.0, 119.0, 119....",water heater,2020-11-03 23:46:31.751,298.257732


In [56]:
(pd.to_datetime('2020-11-03 00:28:50.575000'))> pd.to_datetime('2020-11-04 00:28:50.575000')

False

In [61]:
import datetime

In [69]:
pd.to_datetime('2020-11-03 00:28:50.575000') + datetime.timedelta(seconds=3)

Timestamp('2020-11-03 00:28:53.575000')